In [16]:
# Library imports
import numpy as np

In [17]:
# Hidden Layers
# Dense
class Layer_Dense:
    # Layer initialization
    # randomly initialize weights and set biases to zero
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))


    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)


In [18]:
# Activation Functions
# Included here are the functions for both the forward and backward pass

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [19]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and grou truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        # Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate Binary Cross Entropy
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        # Gradient of BCE loss
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = y_pred.shape[0]

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values
        # Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # Use the first sample to count them
        labels = len(dvalues[0])

        # Check if labels are sparse, turn them into one-hot vector values
        # the eye function creates a 2D array with ones on the diagonal and zeros elsewhere
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate the gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


In [20]:
class Optimizer_SGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate

    def update_params(self, layer):
        layer.weights -= self.current_learning_rate * layer.dweights
        layer.biases -= self.current_learning_rate * layer.dbiases


class Optimizer_SGD_Decay:
    def __init__(self, learning_rate=0.01, decay=0.0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))

    def update_params(self, layer):
        layer.weights -= self.current_learning_rate * layer.dweights
        layer.biases -= self.current_learning_rate * layer.dbiases

    def post_update_params(self):
        self.iterations += 1


class Optimizer_Momentum:
    def __init__(self, learning_rate=0.01, decay=0.0, momentum=0.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.momentum = momentum
        self.iterations = 0

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_momentums'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)

        weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
        bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases

        layer.weight_momentums = weight_updates
        layer.bias_momentums = bias_updates

        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1


class Optimizer_AdaGrad:
    def __init__(self, learning_rate=0.01, decay=0.0, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.epsilon = epsilon
        self.iterations = 0

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        layer.weight_cache += layer.dweights ** 2
        layer.bias_cache += layer.dbiases ** 2

        layer.weights -= self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases -= self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

In [21]:
# Spiral Data
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples = 100, classes = 3)

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [22]:
# Iris Dataset
# From the scikit-learn library
# from sklearn.datasets import load_iris
# iris = load_iris()
# X = iris.data # Features
# y = iris.target # Target labels

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [23]:
# Neural Network initialization
# Create a Dense Layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Make sure you check the shape of the features, in order to adjust the input size of the first layer
# dense1 = Layer_Dense(4, 3)

# Create a ReLU activation for the first Dense layer
activation1 = Activation_ReLU()

# Create a 2nd dense layer with 3 input and 3 output values
dense2 = Layer_Dense(3, 3)

# Create a Softmax activation for the 2nd Dense layer
activation2 = Activation_Softmax()

# Create a loss function
loss_function = Loss_CategoricalCrossEntropy()

# Create the optimizer
optimizer = Optimizer_SGD()

In [24]:
# Perform a forward pass of our training data
# give the input from the dataset to the first layer
dense1.forward(X)

# Activation function
activation1.forward(dense1.output)

# Pass on the 2nd layer
dense2.forward(activation1.output)

activation2.forward(dense2.output)

# Calculate the loss
loss_function.forward(activation2.output, y)

# Check the model's performance
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

# Print the accuracy
print('acc:', accuracy)

acc: 0.3233333333333333


In [25]:
# Perform a backward pass of our training data
# From loss to 2nd softmax activation
loss_function.backward(activation2.output, y)
dvalues = loss_function.dinputs # Gradient of the loss w.r.t softmax output

print(dvalues.shape)
# print(dvalues)

# From 2nd softmax to 2nd dense layer
activation2.backward(dvalues)
# From 2nd dense layer to 1st ReLU activation
dense2.backward(activation2.dinputs)

# From 1st ReLU activation to 1st dense layer
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

(300, 3)


In [26]:
# Check the gradient values of the weights and biases of the established layers
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)


# Update the weights and biases
optimizer.update_params(dense1)
optimizer.update_params(dense2)

[[ 3.78280087e-04 -1.17717912e-04 -9.92780789e-05]
 [-3.74141214e-04 -3.92754013e-05 -1.84885849e-04]]
[[ 6.66974519e-04 -2.43168809e-04  5.76181399e-05]]
[[-1.88699700e-04  1.45459140e-04  4.32405597e-05]
 [-1.54472284e-05 -2.34934678e-05  3.89406962e-05]
 [ 6.42060657e-05  2.46803163e-04 -3.11009228e-04]]
[[-9.64629443e-06  1.65100031e-05 -6.86370865e-06]]


In [27]:
print("\n" + "="*80)
print("WEIGHT UPDATE - 1000 EPOCHS")
print("="*80 + "\n")

dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 64)
activation2 = Activation_ReLU()
dense3 = Layer_Dense(64, 3)
activation3 = Activation_Softmax()

loss_function = Loss_CategoricalCrossEntropy()
optimizer = Optimizer_SGD(learning_rate=0.1)

for epoch in range(1000):
    # ===== FORWARD PASS (FP) =====
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    loss = loss_function.calculate(activation3.output, y)
    predictions = np.argmax(activation3.output, axis=1)
    y_true = y if y.ndim == 1 else np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y_true)

    # ===== BACKPROPAGATION (BP) =====
    loss_function.backward(activation3.output, y)
    activation3.backward(loss_function.dinputs)
    dense3.backward(activation3.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # ===== WEIGHT UPDATE =====
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.update_params(dense3)

    if epoch % 100 == 0:
        print(f'Epoch: {epoch:4d} | Accuracy: {accuracy:.3f} | Loss: {loss:.4f}')

print(f'Epoch: 1000 | Accuracy: {accuracy:.3f} | Loss: {loss:.4f}')
print("="*80)
print(f"\nFinal Result - Epoch: 1000 | Accuracy: {accuracy:.3f} | Loss: {loss:.4f}")
print("="*80)



WEIGHT UPDATE - 1000 EPOCHS

Epoch:    0 | Accuracy: 0.337 | Loss: 1.0986
Epoch:  100 | Accuracy: 0.417 | Loss: 1.0986
Epoch:  200 | Accuracy: 0.440 | Loss: 1.0986
Epoch:  300 | Accuracy: 0.460 | Loss: 1.0986
Epoch:  400 | Accuracy: 0.443 | Loss: 1.0986
Epoch:  500 | Accuracy: 0.417 | Loss: 1.0986
Epoch:  600 | Accuracy: 0.427 | Loss: 1.0986
Epoch:  700 | Accuracy: 0.423 | Loss: 1.0986
Epoch:  800 | Accuracy: 0.417 | Loss: 1.0986
Epoch:  900 | Accuracy: 0.420 | Loss: 1.0986
Epoch: 1000 | Accuracy: 0.427 | Loss: 1.0986

Final Result - Epoch: 1000 | Accuracy: 0.427 | Loss: 1.0986


In [28]:
print("\n" + "="*80)
print("TRAINING WITH LEARNING RATE DECAY - 1000 EPOCHS")
print("="*80 + "\n")

dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 64)
activation2 = Activation_ReLU()
dense3 = Layer_Dense(64, 3)
activation3 = Activation_Softmax()

loss_function = Loss_CategoricalCrossEntropy()
optimizer_decay = Optimizer_SGD_Decay(learning_rate=0.1, decay=1e-3)

decay_results = []

for epoch in range(1000):
    optimizer_decay.pre_update_params()

    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    loss = loss_function.calculate(activation3.output, y)
    predictions = np.argmax(activation3.output, axis=1)
    y_true = y if y.ndim == 1 else np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y_true)

    decay_results.append({'epoch': epoch, 'loss': loss, 'accuracy': accuracy})

    loss_function.backward(activation3.output, y)
    activation3.backward(loss_function.dinputs)
    dense3.backward(activation3.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer_decay.update_params(dense1)
    optimizer_decay.update_params(dense2)
    optimizer_decay.update_params(dense3)

    optimizer_decay.post_update_params()

    if epoch % 100 == 0:
        print(f'Epoch: {epoch:4d} | Acc: {accuracy:.3f} | Loss: {loss:.4f} | LR: {optimizer_decay.current_learning_rate:.5f}')

print(f'Epoch: 1000 | Acc: {accuracy:.3f} | Loss: {loss:.4f} | LR: {optimizer_decay.current_learning_rate:.5f}')
print("="*80)
print(f"Learning Rate Decay - Final Accuracy: {accuracy:.3f}, Final Loss: {loss:.4f}")
print("="*80)


TRAINING WITH LEARNING RATE DECAY - 1000 EPOCHS

Epoch:    0 | Acc: 0.283 | Loss: 1.0986 | LR: 0.10000
Epoch:  100 | Acc: 0.417 | Loss: 1.0986 | LR: 0.09091
Epoch:  200 | Acc: 0.430 | Loss: 1.0986 | LR: 0.08333
Epoch:  300 | Acc: 0.433 | Loss: 1.0986 | LR: 0.07692
Epoch:  400 | Acc: 0.433 | Loss: 1.0986 | LR: 0.07143
Epoch:  500 | Acc: 0.420 | Loss: 1.0986 | LR: 0.06667
Epoch:  600 | Acc: 0.430 | Loss: 1.0986 | LR: 0.06250
Epoch:  700 | Acc: 0.430 | Loss: 1.0986 | LR: 0.05882
Epoch:  800 | Acc: 0.433 | Loss: 1.0986 | LR: 0.05556
Epoch:  900 | Acc: 0.423 | Loss: 1.0986 | LR: 0.05263
Epoch: 1000 | Acc: 0.423 | Loss: 1.0986 | LR: 0.05003
Learning Rate Decay - Final Accuracy: 0.423, Final Loss: 1.0986


In [29]:
print("\n" + "="*80)
print("TRAINING WITH MOMENTUM - 1000 EPOCHS")
print("="*80 + "\n")

dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 64)
activation2 = Activation_ReLU()
dense3 = Layer_Dense(64, 3)
activation3 = Activation_Softmax()

loss_function = Loss_CategoricalCrossEntropy()
optimizer_momentum = Optimizer_Momentum(learning_rate=0.1, decay=1e-3, momentum=0.9)

momentum_results = []

for epoch in range(1000):
    optimizer_momentum.pre_update_params()

    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    loss = loss_function.calculate(activation3.output, y)
    predictions = np.argmax(activation3.output, axis=1)
    y_true = y if y.ndim == 1 else np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y_true)

    momentum_results.append({'epoch': epoch, 'loss': loss, 'accuracy': accuracy})

    loss_function.backward(activation3.output, y)
    activation3.backward(loss_function.dinputs)
    dense3.backward(activation3.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer_momentum.update_params(dense1)
    optimizer_momentum.update_params(dense2)
    optimizer_momentum.update_params(dense3)

    optimizer_momentum.post_update_params()

    if epoch % 100 == 0:
        print(f'Epoch: {epoch:4d} | Acc: {accuracy:.3f} | Loss: {loss:.4f} | LR: {optimizer_momentum.current_learning_rate:.5f}')

print(f'Epoch: 1000 | Acc: {accuracy:.3f} | Loss: {loss:.4f} | LR: {optimizer_momentum.current_learning_rate:.5f}')
print("="*80)
print(f"Momentum - Final Accuracy: {accuracy:.3f}, Final Loss: {loss:.4f}")
print("="*80)



TRAINING WITH MOMENTUM - 1000 EPOCHS

Epoch:    0 | Acc: 0.363 | Loss: 1.0986 | LR: 0.10000
Epoch:  100 | Acc: 0.447 | Loss: 1.0986 | LR: 0.09091
Epoch:  200 | Acc: 0.433 | Loss: 1.0986 | LR: 0.08333
Epoch:  300 | Acc: 0.433 | Loss: 1.0986 | LR: 0.07692
Epoch:  400 | Acc: 0.423 | Loss: 1.0985 | LR: 0.07143
Epoch:  500 | Acc: 0.397 | Loss: 1.0983 | LR: 0.06667
Epoch:  600 | Acc: 0.390 | Loss: 1.0976 | LR: 0.06250
Epoch:  700 | Acc: 0.373 | Loss: 1.0928 | LR: 0.05882
Epoch:  800 | Acc: 0.377 | Loss: 1.0788 | LR: 0.05556
Epoch:  900 | Acc: 0.380 | Loss: 1.0768 | LR: 0.05263
Epoch: 1000 | Acc: 0.413 | Loss: 1.0744 | LR: 0.05003
Momentum - Final Accuracy: 0.413, Final Loss: 1.0744


In [30]:
print("\n" + "="*80)
print("TRAINING WITH ADAPTIVE GRADIENT (AdaGrad) - 1000 EPOCHS")
print("="*80 + "\n")

dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 64)
activation2 = Activation_ReLU()
dense3 = Layer_Dense(64, 3)
activation3 = Activation_Softmax()

loss_function = Loss_CategoricalCrossEntropy()
optimizer_adagrad = Optimizer_AdaGrad(learning_rate=0.1, decay=1e-3)

adagrad_results = []

for epoch in range(1000):
    optimizer_adagrad.pre_update_params()

    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    loss = loss_function.calculate(activation3.output, y)
    predictions = np.argmax(activation3.output, axis=1)
    y_true = y if y.ndim == 1 else np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y_true)

    adagrad_results.append({'epoch': epoch, 'loss': loss, 'accuracy': accuracy})

    loss_function.backward(activation3.output, y)
    activation3.backward(loss_function.dinputs)
    dense3.backward(activation3.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer_adagrad.update_params(dense1)
    optimizer_adagrad.update_params(dense2)
    optimizer_adagrad.update_params(dense3)

    optimizer_adagrad.post_update_params()

    if epoch % 100 == 0:
        print(f'Epoch: {epoch:4d} | Acc: {accuracy:.3f} | Loss: {loss:.4f} | LR: {optimizer_adagrad.current_learning_rate:.5f}')

print(f'Epoch: 1000 | Acc: {accuracy:.3f} | Loss: {loss:.4f} | LR: {optimizer_adagrad.current_learning_rate:.5f}')
print("="*80)
print(f"AdaGrad - Final Accuracy: {accuracy:.3f}, Final Loss: {loss:.4f}")
print("="*80)


TRAINING WITH ADAPTIVE GRADIENT (AdaGrad) - 1000 EPOCHS

Epoch:    0 | Acc: 0.353 | Loss: 1.0986 | LR: 0.10000
Epoch:  100 | Acc: 0.463 | Loss: 0.9533 | LR: 0.09091
Epoch:  200 | Acc: 0.630 | Loss: 0.7945 | LR: 0.08333
Epoch:  300 | Acc: 0.737 | Loss: 0.6360 | LR: 0.07692
Epoch:  400 | Acc: 0.773 | Loss: 0.5385 | LR: 0.07143
Epoch:  500 | Acc: 0.800 | Loss: 0.4775 | LR: 0.06667
Epoch:  600 | Acc: 0.827 | Loss: 0.4307 | LR: 0.06250
Epoch:  700 | Acc: 0.840 | Loss: 0.3902 | LR: 0.05882
Epoch:  800 | Acc: 0.873 | Loss: 0.3583 | LR: 0.05556
Epoch:  900 | Acc: 0.880 | Loss: 0.3346 | LR: 0.05263
Epoch: 1000 | Acc: 0.867 | Loss: 0.3159 | LR: 0.05003
AdaGrad - Final Accuracy: 0.867, Final Loss: 0.3159
